In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, PrecisionRecallDisplay, auc, RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyClassifier


## Precision-Recall Curves - For When ROC Fails

Okay, hopefully you've all seen and get ROC-AUC by now. The ROC-AUC is a great tool for assessing your model's overall performance and for finding the right threshold for a clasification model. But!  It does have some limitations. Let's explore that.

#### First thing, getting a dataset

In [ ]:
data = sns.load_dataset('taxis')

In [ ]:
data.color.value_counts()

Okay, here's a dataset dealing with various taxi rides. Our client is a graphic designer who wants to comparing yellow and green cabs. For some reason, known as 'I need an example', they want to try to predict the color of a cab based on other factors of a cab ride. Since I'm just doing an example to show metrics and I'm not trying to incorperate all the data, I'm going to drop some columns that would take too much preprocessing, just for example. 

In [ ]:
data.info()

In [ ]:
# dropping columns I don't want to deal with. 
# I don't feel like dealing with the time, and the pickup_zone and dropoff_zone both have a ton of values. 
data_dropped = data.drop(['pickup', 'dropoff','pickup_zone','dropoff_zone'], axis = 1)

# dropping nans
data_drop_nan = data_dropped.dropna()

# creating data and target dataframes
data_X = data_drop_nan.drop('color', axis = 1)
target = data_drop_nan.color

# I'm going to relabel the target column into binary. This is won't cause data leak issues. 
# I am leaving scaling and OHE for after the tts of course.
target_binary = target.replace(to_replace = ['yellow', 'green'], value = [0,1])

# train test split
# I'm going to stratify based on target because there's *some* imbalance. 
X_train, X_test, y_train, y_test = train_test_split(data_X, target_binary, stratify = target_binary, random_state=14)

In [ ]:
print('y train ratio = '+ str(y_train.value_counts()[0]/y_train.value_counts()[1]))
print('y test ratio = '+ str(y_test.value_counts()[0]/y_test.value_counts()[1]))

The ratio here is around 5.5 to 1, which is fairly imbalanced. Let's see what happens when we run a pretty simple model on it. First, preprocessing.

In a bit we're going to cover ColumnTransformer and Pipelines that will make these steps quicker, for now I'm going to fly through this, I just need a simple model so I can show off AUC.

In [ ]:
ohe1 = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
scaler1 = StandardScaler()

In [ ]:
columns_for_OHE = ['payment', 'pickup_borough', 'dropoff_borough']
columns_for_scaling = ['distance', 'fare', 'tip', 'tolls', 'total']
# passengers is weird here, it's possible to think of them as just a numeric feature to be scaled, 
# but also to see it as an ordinal category. In this case I'm going to not scale it, but in a real project
# I might test both versions to see if it changes the model. 


#Fit the ohe encoder on the categorical data:
ohe1.fit(X_train[columns_for_OHE])

#Transform the training data
encoded_columns = pd.DataFrame(ohe1.transform(X_train[columns_for_OHE]), 
                               columns = [ohe1.get_feature_names_out()], 
                               index = X_train.index)

#Fit the scaler on the continuous numeric variables
scaler1.fit(X_train[columns_for_scaling])

#Transform using the scaler
scaled_columns = pd.DataFrame(scaler1.transform(X_train[columns_for_scaling]), 
                              columns = [scaler1.feature_names_in_], 
                              index = X_train.index)


display(encoded_columns.head())
display(scaled_columns.head())

In [ ]:
# Joining the two transformed data frame
X_train_encscaled = scaled_columns.join(encoded_columns)

# And now adding the one column I didn't encode or scale. 
# Again, ColumnTransformer and Pipelines makes this much nicer.
X_train_encscaled['passengers']= X_train.passengers

X_train_encscaled 

Finally done with that! Let's put this in a model, get an AUC score, and maybe I can talk about Precision-Recall Curves.

In [ ]:
baselinemodel = DummyClassifier()
lrmodel1 = LogisticRegression(random_state = 14)

baselinemodel.fit(X_train_encscaled, y_train)
lrmodel1.fit(X_train_encscaled, y_train)

In [ ]:
def pseudopipeline(Xdata, fitenc, fitscaler):
    '''
    Takes in a test X data set, fit ohe and fit scaler.
    Returns a properly set up X_test to feed to fit models.
    '''
    pipe_encoded_columns = pd.DataFrame(fitenc.transform(Xdata[columns_for_OHE]), 
                               columns = [fitenc.get_feature_names_out()], 
                               index = Xdata.index)
    pipe_scaled_columns = pd.DataFrame(fitscaler.transform(Xdata[columns_for_scaling]), 
                              columns = [fitscaler.feature_names_in_], 
                              index = Xdata.index)
    

    
    
    X_df_encscaled = pipe_scaled_columns.join(pipe_encoded_columns)
    X_df_encscaled['passengers']= Xdata.passengers
    return X_df_encscaled

### For this example, I will now drop many of the 'green' taxis from the y test data to simulate a very imbalanced class problem

In [ ]:
y_green = y_test.loc[y_test== 1].sample(frac=.1,  random_state=14)
#X_test[]
print(len(y_green))
y_yellow = y_test.loc[y_test == 0]
y_yellow
new_y_test = pd.DataFrame(pd.concat([y_green, y_yellow]))
print(len(new_y_test))
new_test_df = new_y_test.join(X_test)


In [ ]:
# Once you get here, don't run cells above this, we're using new values for y_test and X_test
# Unless you run the whole thing back from the top

y_test = new_test_df.color

X_test = new_test_df.drop(labels = 'color', axis = 1)

#I'll run my X_test data into my pseudopipeline

X_test_enscaled = pseudopipeline(X_test, ohe1, scaler1)

In [ ]:
#Generating predictions with default threshold
y_pred_dummy = baselinemodel.predict(X_test_enscaled)
y_pred_1 = lrmodel1.predict(X_test_enscaled)



#Generating my predictions as probabilities
y_pred_proba_dummy = baselinemodel.predict_proba(X_test_enscaled)
y_pred_proba_1 = lrmodel1.predict_proba(X_test_enscaled)

#
def confusiondisplay(y_true, y_pred):
    cm = ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
    return cm

In [ ]:
y_pred_proba_dummy

In [ ]:
confusiondisplay(y_test, y_pred_dummy)
print('DummyConfusion Matrix')

In [ ]:
print('Model 1 Confusion Matrix')
confusiondisplay(y_test, y_pred_1)

In [ ]:
def plotrocauc(y_true, y_pred_probs, model_name):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
    roc_auc = auc(fpr, tpr)
    display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
    display.plot()
    auc_score = roc_auc_score(y_true, y_pred_probs)
    print('ROC - AUC score for ' + model_name + ' is ' + str(auc_score))
    

In [ ]:
plotrocauc(y_test, y_pred_proba_1[ :,1], 'model1')


In [ ]:
plotrocauc(y_test, y_pred_proba_dummy[ :,1], 'Dummy')

Okay, so what's the deal with this PR curve?  Where a ROC curve charts true and false positive *rates*, the Precision-Recall curve charts... precision and recall. 

Remember that Precision is the same as the number of true positives divided by the total number of both true and false positives. This is the same as the 'TPR' in the ROC-AUC metric.  A high precision means that you are rarely missing positive cases. As we've seen, you can get more of the true positive by trading off, with a corresponding growth in false negatives. 

So the real difference with PR-AUC and ROC-AUC is Recall, which divides your total number of true positive guesses by the overall number of positive guesses. To increase recall, you need to weed out false negatives. 

|                   | Guessed True  | Guessed False  |
| --------------    | --------------|--------------- |
| **Actually True** |        TP     |        FN      |
| **Actually False**|        FP     |        TN      |


Precision Recall can be a more useful metric in cases of imbalanced datasets. An ROC-AUC on an imbalanced dataset might 'get away with' an inflated TPR and FPR if the imbalance leads to a relatively 'good' rate of true negatives.


Consider a case like this:

TP=9, FN=1, FP=90, TN=900 

|                    | Guessed Green     | Guessed Yellow     |
| --------------     | ------------------|------------------- |
| **Actually Green** |        9          |        1           |
| **Actually Yellow**|       90          |       900          |

Now, how well do we think this model worked? Well, if we're trying to find Greens, we did pretty well. And while we had a lot of yellows in the green guesses, we were fairly discriminating. Mostly, we were able to class the Yellows into the right column.


Let's see how this would reflect in our ROC-AUC curve at whatever threshold we're using. In this case our FPR is **FP / (FP + TN)** or 1/(1+900). The large amount of 'True Yellows' in the sample lead this to have a 'good' FPR score, maybe better than we might feel is justified. The large number of 'Actually Yellow' guesses will 'stick' the curve to the left. On the other hand, the TPR (**TP/(TP+FP)**)is less good at (9/9+90). That's obviously not good, but it is skewed by the exaggerated number of 'Actually Yellow' responses in the dataset. Although we could use the ROC to find a good threshold with our model, the ROC-AUC score in general will be skewed by this class imbalance.

Let's compare that to the Precision - Recall metric. The Precision is the same as the TPR **(TP/(TP+FP)**), but Recall is **TP / (TP + FN)** or 9/(9+1), or 90%. That's not bad. The PR-Curve 'looks at' the upper left, upper right, and lower left cells, but doesn't look at the lower right. In other words, **TN** is not accounted for in the PR-AUC. 

In [ ]:
def plotprauc(fitestimator, xtest, y_true, model_name, y_pred_probs):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_probs)
    PrecisionRecallDisplay.from_estimator(estimator = fitestimator, X = xtest, y = y_true, name = model_name)
    auc_score = auc(recall, precision)
    print('PR - AUC score for ' + model_name + ' is ' + str(auc_score))
    #disp.plot()


In [ ]:
plotprauc(lrmodel1, X_test_enscaled, y_test,  'Model 1', y_pred_proba_1[ :,1])

In [ ]:
plotprauc(baselinemodel, X_test_enscaled, y_test,'Dummy', y_pred_proba_dummy[ :,1])